## Ayudantía 09:  I/O y Serialización

#### Autores: @Alonsinho1 & @dchahuan


Puedes evaluar esta ayudantía [aquí]


En esta ayudantía tiene como proposito que interioricen los conpectios de I/O y Serialización. Para esto haremos un breve repaso y unos cuantos ejercicios que les permitira visualizar la materia.

### ¿Qué son los bits y los bytes?:

Un __byte__ es la estructura basica para guardar datos computacionalmente. Este esta compuesto de 8 __bits__. Cada bit es un numero que puede tener como valor 1 o 0.

Tal como medimos la naturaleza mediante unidades de medida (gramos, litros, Newton, etc.), el byte lo utilizamos para medir el tamaño de los archivos de un computador.

__Para mayor información de como el computador interpreta esta info, puede tomar el curso IIC2343 Arquitectura de Computadores__

### ¿Qué significa serializar?:

Serializar es transformar un objeto en una secuencia o serie de bytes. Esto permite tener la información en un estado de forma persistente, lo que sirve para enviar el objeto a otros computadores y programas.

### Formatos de seralización

#### JSON

Este formato es limitado, pues solamente permite serializar __algunos__ objetos específicos como strings y diccionarios. Sin embargo, es legible por humanos.

#### Pickle

Pickle es un formato específico de Python para serializar. Una de las ventajas es que este puede serializar __cualquier__ objeto de python, sin embargo no puede ser leído por otros lenguajes de programación o por humanos.

Como este no puede ser leido por humanos, es posible incluír código malicioso en un programa que utiliza pickle, por lo que __NO__ deberías ejecutar código que incluya pickle si es que no confías en su origen.

### Resumen de JSON Vs Pickle

![Resumen](imgs/Capture.png)

### Bytes en IIC2233

En este curso se les pedira que puedan leer, editar y guardar estos tipos de datos, al igual como lo han hecho con los strings, diccionarios, etc...

#### Open()

La función open() les permitira leer cualquier archivo sin que estos necesariamente sean de texto. Cual es la ventaja de usar open() directamente?

In [26]:
with open('datos.bin', 'rb') as byte_file:
    datos = byte_file.read()
    print(datos)
    print(type(datos))
    mis_bytes = bytearray(datos)
    print(type(mis_bytes))
    
with open('datos2.bin', 'wb') as byte_file:
    byte_file.write(datos)
    
with open('datos2.bin', 'rb') as byte_file:
    print(byte_file.read())
    mis_bytes = bytearray(byte_file.read())

b'\x00\x03\x00\x03\x04\x05\x06'
<class 'bytes'>
<class 'bytearray'>
b'\x00\x03\x00\x03\x04\x05\x06'


#### bytes() v/s bytearray()

En Python existen dos metodos para definir la estrucutra de byte. Estos son, _bytes()_ y _bytearray()_

Estos métodos permitirán contener bytes o transformar otros tipos de datos a un objeto de tipo bytes.

La diferencia entre bytes y bytearrays es la misma que entre _tuples_ y _lists_:

- _bytes_ y _tuples_ son __inmutables__. Mientras que,

- _bytearrays_ y _lists_ son __mutables__.

In [32]:
datos = bytes([1, 12, 255]) #transformo una lista de ints (0 <= x <= 255)
print(datos)

inversos = bytearray()
for i in range(len(datos)-1, -1, -1):
    inversos.append(datos[i])

print(inversos)

for i in inversos:
    print(i)

b'\x01\x0c\xff'
bytearray(b'\xff\x0c\x01')
255
12
1


#### Conversión de bytes a números enteros


In [15]:
print(int.from_bytes(b'\x01\x01', byteorder='big')) # Utilizaremos el estandar big endian [leer un número de izquierda a derecha]


257


#### Obtener los bits de un byte

In [4]:
un_byte = b'\x7a'

#pasamos de bytes a int
int_byte = int.from_bytes(un_byte, 'big')
print('1.', int_byte)

#transformamos a un string binario
bin_byte = bin(int_byte)
print('2.', bin_byte)

#eliminamos el 0b del formato bin
bin_byte = bin_byte[2:]
print('3.', bin_byte)

#rellenamos con 0 a la izquierda, para ajustar el tamaño a 8 bits.
bits_byte = bin_byte.zfill(8)
print('4.', bits_byte)

1. 122
2. 0b1111010
3. 1111010
4. 01111010


### Ejercicios Propuestos:

¡Han contaminado el recetario de las pizzas del Tini Tamburini! Es por esta razón que han venido a pedirte ayuda mediante este ejercicio propuesto.

Deberás deserializar el archivo **pizzeria.json** que contiene una lista de objetos de clase `Pizza`. Estos objetos tienen el atributo `ingredientes`, el cual ha sido contaminado con elementos que no son comestibles. Para identificarlos, el Tini le entrega un archivo llamado **comestibles.json**, el cual será de ayuda para filtrar los ingredientes mediante el uso un `object_hook`. Finalmente, imprime todas las pizzas utilizando *f-strings*.

In [2]:
import json

class Pizza:
    def __init__(self, nombre, ingredientes, *args, **kwargs):
        self.nombre = nombre
        self.ingredientes = ingredientes
    def __repr__(self):
        return f"Nombre: {self.nombre}, Ingredientes {self.ingredientes}"


def comestibles():
    with open("comestibles.json","rb") as archivo:
        data = json.load(archivo)
    return data

def filtrar_comestibles(diccionario):
    ingredientes_comestibles = comestibles()

    diccionario["ingredientes"] = list(filter(lambda x: x in ingredientes_comestibles, diccionario["ingredientes"]))


    return Pizza(**diccionario)



# Carga las pizzas del archivo pizzeria.json
# y finalmente imprime la lista de pizzas usando f-strings
with open("pizzeria.json","rb") as archivo:
    data = json.load(archivo, object_hook=filtrar_comestibles)
    for pizza in data:
        print(pizza)



Nombre: hawaiana, Ingredientes ['pinia', 'queso', 'jamon']
Nombre: vegetariana, Ingredientes ['queso', 'aceitunas', 'champiniones']
Nombre: machas a la parmesana, Ingredientes ['machas', 'queso']
Nombre: pepperoni, Ingredientes ['pepperoni', 'queso']
Nombre: pollo bbq, Ingredientes ['cebolla', 'queso', 'pollo bbq']
Nombre: jamon palmito, Ingredientes ['jamon', 'palmitos', 'queso']
Nombre: white, Ingredientes ['aceitunas', 'tomate']
Nombre: napolitana, Ingredientes ['queso', 'albahaca', 'tomate']
Nombre: queso, Ingredientes ['queso', 'queso']


## Ejercicio 2: `bytearrays` y *context managers*


El archivo **simpatico.bmp** ha sido corrompido con *bytes* no correspondientes. Para limpiarlo, deberás implementar el siguiente algoritmo:

1. Abrir el archivo **malvado.mal** en modo lectura de *bytes*.
2. Obtener *chunks* de 8 bytes.
3. Encontrar el máximo de esos 8 *bytes*.
4. Eliminar todos los *bytes* con ese valor dentro del *chunk*.
5. Repetir el proceso con el resto de los *bytes*.
6. Escribir el archivo **simpatico.bmp** con los bytes obtenidos.

**Sobre los pasos 2 a 4**: 

- Considerando un *chunk* de 8 *bytes*, con los siguientes valores (paso 2): `4 8 255 15 16 23 255 42`
- Se obtiene el máximo de ese *chunk* de bytes (paso 3): `255`.
- Se eliminan todos los *bytes* con ese valor, resultando: `4 8 15 16 23 42`

In [4]:
# Se abre el archivo
with open('malvado.mal', 'rb') as bytefile:
    byte_read = bytefile.read()
    byte_out = bytearray()
    # Se recorre su contenido en chunks de 8 bytes
    for i in range(0, len(byte_read), 8):
        # Se obtiene un chunk de 8 bytes
        chunk = bytearray(byte_read[i:i+8])
        # Se obtiene el valor más alto
        maximum = max(chunk)
        # Se eliminan todos los bytes con ese valor
        while maximum in chunk:
            chunk.remove(maximum)
        byte_out += chunk

# Se escribe el nuevo archivo
with open('simpatico.bmp', 'wb') as file_out:
    file_out.write(byte_out)

0


### Ejercicio AS04 2020-1

En el enunciado de la actividad salia lo siguiente:

La información del ayudante jefe coordinador se encuentra corrompida en un archivo cuyos bytes fueron
manipulados. Tendrás que reparar dicho archivo para recuperar los datos del ayudante, completando la
función reparar_usuario que se encuentra dentro del archivo reparar_bytes.py.

def reparar_usuario(ruta): 
Esta función recibe la ruta del archivo a reparar, lee el archivo
como bytes y mediante un algoritmo los modifica para escribir los bytes originales en el archivo user_info.bmp. El algoritmo recorre segmentos de 32 bytes contiguos y para cada uno sabes que:

1. El primer byte corresponde a un entero que puede ser 1 o 0, los siguientes 16 bytes pertenecen
al archivo original y los últimos 15 NO pertenecen al archivo original.

2. Si el primer byte es un 1, significa que los siguientes 16 bytes han sido invertidos, mientras que
si es un 0, entonces están en el orden original. Por invertido, se refiere a que si la secuencia es
1 2 3 4, entonces la original es 4 3 2 1.

3. Es necesario extraer los bytes originales al archivo en el orden que corresponda. Luego se avanza
al siguiente segmento de 32 bytes.

Si se concatenan las porciones de bytes extraı́das, respetando el orden de los segmentos de 32 bytes, se
obtiene el contenido completo que se escribe en el archivo de imagen user_info.bmp. Al abrir la imagen,
si se aplicó el algoritmo correctamente, se revelarán los datos del ayudante jefe.

In [5]:
def reparar_imagen(ruta):
    with open(ruta,"rb") as archivo_bytes:
        archivo_final = bytearray()
        data = archivo_bytes.read(32)
        while data:

            data = data[:17]
            
            byte_inicial = data[0]
            data = data[1:]
            
            if byte_inicial == 1:
                data = data[::-1]
            
            archivo_final.extend(data)
            data = archivo_bytes.read(32)
    

    with open("img_reparada.bmp","wb") as archivo_nuevo:
        archivo_nuevo.write(archivo_final)



if __name__ == '__main__':
    try:
        reparar_imagen('imagen_danada.xyz')
        print("Contraseña reparada")
    except Exception as error:
        print(f'Error: {error}')
        print("No has podido obtener la información del Ayudante!")  

Contraseña reparada
